In [1]:
from os.path import dirname, basename

import pandas as pd
import numpy as np

from inpainting.load import load_dataframe
from inpainting.utils import get_paths


In [2]:
dataset = 'DAVIS'
results_dirs = get_paths(f'../results/ii/{dataset}/*')

times_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Benchmark/*/inference_times.csv'):
        times_df = load_dataframe(path)
        times_df['model'] = basename(results_dir)
        times_df['sequence_name'] = basename(dirname(path))
        times_dfs.append(times_df)
times_df = pd.concat(times_dfs)

results_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Evaluation/*/results.csv'):
        results_df = load_dataframe(path)
        results_df['model'] = basename(results_dir)
        sequence_name = basename(dirname(path))
        results_df['sequence_name'] = sequence_name
        results_df['object_sequence_name'] = sequence_name.split('_')[1]
        results_df['object_id'] = int(sequence_name.split('_')[2])
        results_dfs.append(results_df)
results_df = pd.concat(results_dfs)

df = pd.merge(times_df, results_df, on=['model', 'sequence_name', 'frame_id'])
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
df

,frame_id,inference_time,model,sequence_name,mean_squared_error,peak_signal_noise_ratio,structural_similarity,object_sequence_name,object_id
0,0,61.243298,ii_deepfillv1,bear_pigs_001,46.725176,31.435294,0.968047,pigs,1
1,1,61.146751,ii_deepfillv1,bear_pigs_001,104.029925,27.959221,0.963378,pigs,1
2,2,55.534912,ii_deepfillv1,bear_pigs_001,80.026365,29.098473,0.962111,pigs,1
3,3,38.402176,ii_deepfillv1,bear_pigs_001,138.224665,26.724948,0.958946,pigs,1
4,4,39.394978,ii_deepfillv1,bear_pigs_001,157.089327,26.169337,0.962557,pigs,1
...,...,...,...,...,...,...,...,...,...
41260,44,635.715515,ii_regionfill,walking_rallye_001,584.342010,20.464133,0.793267,rallye,1
41261,45,755.281921,ii_regionfill,walking_rallye_001,782.216220,19.197535,0.760024,rallye,1
41262,46,600.637878,ii_regionfill,walking_rallye_001,676.047956,19.831029,0.793386,rallye,1
41263,47,566.722717,ii_regionfill,walking_rallye_001,675.415990,19.835090,0.816619,rallye,1


In [3]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['inference_time'] = df['inference_time']
metrics['mean_squared_error'] = df['mean_squared_error']
metrics['peak_signal_noise_ratio'] = df['peak_signal_noise_ratio']
metrics['structural_similarity'] = df['structural_similarity']
metrics.groupby('model').mean()

,inference_time,mean_squared_error,peak_signal_noise_ratio,structural_similarity
model,,,,
ii_deepfillv1,52.876028,137.478713,30.995413,0.957047
ii_deepfillv1_flownet2,349.900539,83.569036,35.059311,0.972399
ii_deepfillv1_pwcnet,278.251775,82.882268,35.034977,0.972563
ii_deepfillv2,32.362494,117.310768,30.871036,0.957857
ii_kernelfill,277.737633,111.953052,32.777999,0.965099
ii_pconvunet,24.212770,291.961397,27.448398,0.947243
ii_regionfill,111.282765,78.546540,33.829873,0.966292
